In [1]:
!pip install requests beautifulsoup4

  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.5-py3-none-any.whl.metadata (4.7 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
Using cached soupsieve-2.5-py3-none-any.whl (36 kB)


In [85]:
# Import dependencies
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

# 1. Instantiate Model

In [9]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = TFAutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Error while downloading from https://cdn-lfs.huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment/cb7ef0d2a4879de37b6a8824b2c6e213526360d4d89b653d674ff58e220161cb?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27tf_model.h5%3B+filename%3D%22tf_model.h5%22%3B&Expires=1710081779&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMDA4MTc3OX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9ubHB0b3duL2JlcnQtYmFzZS1tdWx0aWxpbmd1YWwtdW5jYXNlZC1zZW50aW1lbnQvY2I3ZWYwZDJhNDg3OWRlMzdiNmE4ODI0YjJjNmUyMTM1MjYzNjBkNGQ4OWI2NTNkNjc0ZmY1OGUyMjAxNjFjYj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=mnhDEBDhMc3wCugAk%7ErzIaWesr8ZTqCuadhJcs5VRjz1mq%7EAxLvTmu1oANeD1Pvz4Vih9ZdKZRB%7EPN5quP-IfdrSe99vw2mIIH5D0JcKGt4Zfb4f3CKd2U%7EkNa15bo3qbORujxNxMfPDs7GaMPB348miDpJAhhSUqqlAM5TsivxHxJZWlw4snLpX5WVlIIBBPRu4XAMYaZ4991%7E%7EojQdTMNxQHUFmPUsp45dFjIIW5xcZnQ-geBsBJVNhZs9Deso0jt-pqb%7EOq3R0c6U1-pGOTF1U4ASBBR0s42SC5air954HMXLH4uZv4

# 2. Encode and Calculate Sentiment

In [58]:
tokens = tokenizer.encode('it\'s great but we can going better', return_tensors='tf') #tf for tensorflow `pt` for pytorch

In [59]:
tokens
# if u can decode this using tokenizer.decode(tokens[0])
# tokenizer.decode(tokens[0])

<tf.Tensor: shape=(1, 11), dtype=int32, numpy=
array([[  101, 10197,   112,   161, 11838, 10502, 11312, 10743, 17010,
        16197,   102]], dtype=int32)>

In [60]:
result = model(tokens)

In [61]:
result

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 5), dtype=float32, numpy=
array([[-2.3674352 , -0.7867989 ,  1.3345441 ,  1.5050346 ,  0.20273374]],
      dtype=float32)>, hidden_states=None, attentions=None)

In [62]:
result.logits

<tf.Tensor: shape=(1, 5), dtype=float32, numpy=
array([[-2.3674352 , -0.7867989 ,  1.3345441 ,  1.5050346 ,  0.20273374]],
      dtype=float32)>

In [63]:
#Convert value into review (add +1 because start from 0)
int(np.argmax(result.logits))+1

4

# 3. Collect Reviews
- Wek will extract from `https://www.yelp.com/biz/social-brew-cafe-pyrmont`

In [70]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [84]:
reviews

['The food was delicious. The ricotta pancakes were light and tasty.  The cream brûlée French toast was a fruit lovers delight. There was fresh strawberries, blueberries, passion fruit and custard. No syrup required. And crispy kale as a side ordered with eggs. Worth the walk over the Pyrmont bridge.',
 "Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!",
 "Six of us met here for breakfast before our walk to Manly. We were enjoying visiting with each other so

# 4. Load Reviews into DataFrame and Score

## Using Pandas

In [86]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [87]:
df

,review
0,The food was delicious. The ricotta pancakes w...
1,Very cute coffee shop and restaurant. They hav...
2,Six of us met here for breakfast before our wa...
3,Great place with delicious food and friendly s...
4,"Great service, lovely location, and really ama..."
5,Great food amazing coffee and tea. Short walk ...
6,Ricotta hot cakes! These were so yummy. I ate ...
7,It was ok. Had coffee with my friends. I'm new...
8,We came for brunch twice in our week-long visi...
9,I came to Social brew cafe for brunch while ex...


In [100]:
# Function to predict Sentiment
def sentiment_score(review):
    token = tokenizer.encode(review, return_tensors='tf')
    result = model(token)
    return int(np.argmax(result.logits))+1

In [112]:
# Make new column and loop for entire row using `lambda` function
# Limit into maximum token into 512
df['sentiment'] = df['review'].apply(lambda x:sentiment_score(x[:512]))

In [113]:
df

,review,sentiment
0,The food was delicious. The ricotta pancakes w...,4
1,Very cute coffee shop and restaurant. They hav...,4
2,Six of us met here for breakfast before our wa...,4
3,Great place with delicious food and friendly s...,5
4,"Great service, lovely location, and really ama...",5
5,Great food amazing coffee and tea. Short walk ...,5
6,Ricotta hot cakes! These were so yummy. I ate ...,5
7,It was ok. Had coffee with my friends. I'm new...,3
8,We came for brunch twice in our week-long visi...,4
9,I came to Social brew cafe for brunch while ex...,5


## Other Way (Just using for loop)

In [114]:
sentiments = []
for comment in reviews:
    sentiment = sentiment_score(comment[:512])
    sentiments.append(sentiment)

In [115]:
for idx, comment in enumerate(reviews):
    print(f'{sentiments[idx]} - {comment}')

4 - The food was delicious. The ricotta pancakes were light and tasty.  The cream brûlée French toast was a fruit lovers delight. There was fresh strawberries, blueberries, passion fruit and custard. No syrup required. And crispy kale as a side ordered with eggs. Worth the walk over the Pyrmont bridge.
4 - Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!
4 - Six of us met here for breakfast before our walk to Manly. We were enjoying visiting with each other 

In [116]:
df_result = pd.DataFrame({'review': np.array(reviews), 'sentiment': np.array(sentiments)})

In [117]:
df_result

,review,sentiment
0,The food was delicious. The ricotta pancakes w...,4
1,Very cute coffee shop and restaurant. They hav...,4
2,Six of us met here for breakfast before our wa...,4
3,Great place with delicious food and friendly s...,5
4,"Great service, lovely location, and really ama...",5
5,Great food amazing coffee and tea. Short walk ...,5
6,Ricotta hot cakes! These were so yummy. I ate ...,5
7,It was ok. Had coffee with my friends. I'm new...,3
8,We came for brunch twice in our week-long visi...,4
9,I came to Social brew cafe for brunch while ex...,5
